In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
# from sklearn.feature_selection import SelectFromModel
# from sklearn.linear_model import LassoCV
import numpy as np

In [3]:
dataPath = "../../data/MLData/inst_feature_matrix_non_syn_20180329.csv"

In [4]:
df = pd.read_csv(dataPath)
df = df.set_index("isolate")
X_df = df.drop(labels = ["pbr_res"], axis = 1)
X = X_df.values
Y_df = df["pbr_res"]
Y = Y_df.values

In [79]:
# feature selection union
# unused right now
features = []
# features.append(('pca', PCA(n_components=3)))
# features.append(('select_best', SelectKBest(k=6)))
# clf = LassoCV()

# Set a minimum threshold of 0.25
features.append(SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False),
                                threshold=0.25, prefit = False))
feature_union = features#FeatureUnion(features)

In [80]:
modelDict = {}

In [81]:
# create pipeline
estimators_LR = []
estimators_LR.append(('feature_union', feature_union))
estimators_LR.append(('logistic', LogisticRegression()))
# estimators.append(models)
modelDict["logistic"] = Pipeline(estimators_LR)

TypeError: All intermediate steps should be transformers and implement fit and transform. '[SelectFromModel(estimator=LinearSVC(C=0.01, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0),
        norm_order=1, prefit=False, threshold=0.25)]' (type <class 'list'>) doesn't

In [30]:
# create pipeline
estimators_RF = []
# estimators.append(('feature_union', feature_union))
estimators_RF.append(('RFC', RandomForestClassifier()))
# estimators.append(models)
modelDict["randomForest"] = Pipeline(estimators_RF)

In [31]:
# create pipeline
estimators_SVC = []
# estimators.append(('feature_union', feature_union))
estimators_SVC.append(('SVC', SVC()))
# estimators.append(models)
modelDict["SVC"] = Pipeline(estimators_SVC)

In [32]:
# create pipeline
estimators_GBTC = []
# estimators.append(('feature_union', feature_union))
estimators_GBTC.append(('GBTC', GradientBoostingClassifier()))
# estimators.append(models)
modelDict["gradientBosting"] = Pipeline(estimators_GBTC)

In [56]:
clf = SVC(kernel='linear', C=1)
cross_val_score(clf, X, Y, cv=kfold,  scoring=scoring)

ValueError: scoring value should either be a callable, string or None. ['accuracy', 'f1'] was passed

In [62]:
seed = 7
kfold = KFold(n_splits=10, random_state=seed)
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1' : make_scorer(f1_score)}
scoring = ["accuracy", "f1", "precision", "recall"]
metrics = {}
for modelName, model in modelDict.items():
    results = cross_validate(model, X, Y, cv=kfold,  scoring=scoring, )
#     results = cross_val_score(estimator=model,
#                               X=features,
#                               y=labels,
#                               cv=kfold,
#                               scoring=scoring)
#     break
    for idx, metric in enumerate(scoring):
        metrics[metric] = results["test_"+metric].mean()
#     accMean = results["test_"+scoring[idx]].mean()
    print("Metrics for {}".format(modelName))
    print(metrics)
    print("\n")
#     break
# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(model, X, Y, cv=kfold)
# print(results.mean())

/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Metrics for SVC
{'f1': 0.47853490698318285, 'accuracy': 0.36499999999999994, 'recall': 0.8, 'precision': 0.35}


Metrics for randomForest
{'f1': 0.44041148325358853, 'accuracy': 0.525, 'recall': 0.42773612008906126, 'precision': 0.5381196581196581}


Metrics for logistic
{'f1': 0.5479076587900117, 'accuracy': 0.585, 'recall': 0.5793830352653881, 'precision': 0.6041592394533571}


Metrics for gradientBosting
{'f1': 0.569554028901855, 'accuracy': 0.5850000000000001, 'recall': 0.589656683186095, 'precision': 0.6127168746286393}




In [59]:
results

/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/oliver/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_f1'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.09365368, 0.08773923, 0.09179783, 0.09282327, 0.09159279,
        0.08882642, 0.08915591, 0.09183574, 0.08691478, 0.08693004]),
 'score_time': array([0.0196836 , 0.01922178, 0.02096891, 0.02029967, 0.01987886,
        0.01960826, 0.01916647, 0.02153134, 0.01918578, 0.01924086]),
 'test_accuracy': array([0.6 , 0.5 , 0.15, 0.  , 0.65, 0.45, 0.35, 0.35, 0.3 , 0.3 ]),
 'test_f1': array([0.75      , 0.66666667, 0.        , 0.        , 0.78787879,
        0.62068966, 0.51851852, 0.51851852, 0.46153846, 0.46153846]),
 'train_accuracy': array([0.52777778, 0.53888889, 0.53888889, 0.51666667, 0.52222222,
        0.54444444, 0.55555556, 0.55555556, 0.56111111, 0.56111111]),
 'train_f1': array([0.69090909, 0.70036101, 0.22429907, 0.        , 0.68613139,
        0.70503597, 0.71428571, 0.71428571, 0.71886121, 0.71886121])}

In [42]:
results

array([0.75, 0.35, 0.7 , 0.6 , 0.4 , 0.55, 0.65, 0.5 , 0.7 , 0.85])

In [39]:
model = Pipeline(estimators_GBTC)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.5900000000000001


In [36]:
model

Pipeline(memory=None,
     steps=[('GBTC', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])

In [8]:
np.mean(Y)

0.535

In [33]:
modelDict

{'SVC': Pipeline(memory=None,
      steps=[('SVC', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))]), 'gradientBosting': Pipeline(memory=None,
      steps=[('GBTC', GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               presort='auto', random_state=None, subsample=1.0, verbose=0,
               warm_start=False))]), 'logistic': Pipeline(memory=None,
      steps=[('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max